<a href="https://colab.research.google.com/github/luisjfeliu/python-flower-species-image-classifier/blob/main/Image%20Classifier%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare the workspace

In [1]:
# Google Colab
!git lfs install
!git clone https://github.com/luisjfeliu/python-flower-species-image-classifier.git
%cd python-flower-species-image-classifier
!ls

!pip -q install --upgrade pip
!pip -q install numpy matplotlib
!pip -q install torch torchvision

Git LFS initialized.
Cloning into 'python-flower-species-image-classifier'...
remote: Enumerating objects: 8522, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 8522 (delta 1), reused 4 (delta 1), pack-reused 8516 (from 4)
Receiving objects: 100% (8522/8522), 417.90 MiB | 25.65 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (8202/8202), done.
/content/python-flower-species-image-classifier
 assets		    flower_classifier.pth	      LICENSE	   train.py
 cat_to_name.json   flowers			      predict.py
 environment.yml   'Image Classifier Project.ipynb'   README.md
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 38.0 MB/s eta 0:00:00


In [2]:
# Check torch version and CUDA status if GPU is enabled.
import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available()) # Should return True when GPU is enabled.

torch: 2.9.0+cu126
cuda available: True


# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications.

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below.

<img src='https://github.com/luisjfeliu/python-flower-species-image-classifier/blob/main/assets/Flowers.png?raw=1' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [17]:
# Imports here
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision.transforms import v2
from torchvision.models import list_models, resnet50, ResNet50_Weights
import json, time, copy, numpy as np, matplotlib

print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("cuda available:", torch.cuda.is_available()) # Should return True when GPU is enabled.
print("numpy:", np.__version__)

torch: 2.9.0+cu126
torchvision: 0.24.0+cu126
cuda available: True
numpy: 2.0.2


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz).

If you do not find the `flowers/` dataset in the current directory, **/workspace/home/aipnd-project/**, you can download it using the following commands.

**Method 1**
```bash
!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
!unlink flowers
!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers
```


**Method 2**
```bash
!cp -r /data/ .
```

## Data Description
The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.


In [4]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [16]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transform = v2.Compose([
        v2.RandomResizedCrop(size=224),
        v2.RandomVerticalFlip(),
        v2.RandomHorizontalFlip(),
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
valid_transform = v2.Compose([
        v2.Resize(size=256),
        v2.CenterCrop(size=224),
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
test_transform = valid_transform

# TODO: Load the datasets with ImageFolder
train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transform)
valid_dataset = torchvision.datasets.ImageFolder(valid_dir, valid_transform)
test_dataset = torchvision.datasets.ImageFolder(test_dir, test_transform)


# TODO: Using the image datasets and the trainforms, define the dataloaders
num_workers = 2
batch_size=32
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=num_workers)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=num_workers)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=num_workers)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [6]:
cat_to_name = {}
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

print(cat_to_name)

num_classes = len(cat_to_name)
print(f"number of flower classes: {num_classes}")

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active.

## Note for Workspace users:
If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [7]:
all_models = list_models()
classification_models = list_models(module=torchvision.models)
print(classification_models)

['alexnet', 'convnext_base', 'convnext_large', 'convnext_small', 'convnext_tiny', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'efficientnet_v2_l', 'efficientnet_v2_m', 'efficientnet_v2_s', 'googlenet', 'inception_v3', 'maxvit_t', 'mnasnet0_5', 'mnasnet0_75', 'mnasnet1_0', 'mnasnet1_3', 'mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small', 'regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf', 'regnet_x_400mf', 'regnet_x_800mf', 'regnet_x_8gf', 'regnet_y_128gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf', 'regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'resnext101_32x8d', 'resnext101_64x4d', 'resnext50_32x4d', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'squeezenet1_0

In [8]:
# TODO: Build and train your network
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

for param in model.parameters():
    param.requires_grad = False

num_features = model.fc.in_features

print(num_features)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 147MB/s]


2048


In [9]:
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, num_classes)
)

print(model.fc)

Sequential(
  (0): Linear(in_features=2048, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=512, out_features=256, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=256, out_features=102, bias=True)
)


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model loaded on: {device}")

Model loaded on: cuda


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.fc.parameters())

In [12]:
def train(model, train_loader, loss_fn, optimizer, device):
    """
    Train the model for one epoch.

    Args:
        model: The neural network model to train
        train_loader: DataLoader for training data
        loss_fn: Loss function
        optimizer: Optimization algorithm
        device: Device to run training on (cuda/cpu)

    Returns:
        tuple: (average_loss, accuracy_percentage) - average loss and accuracy percentage
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    num_batches = len(train_loader)

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if batch_idx == 0 or (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == num_batches:
            print(f"Train Batch [{batch_idx + 1}/{num_batches}] | Loss: {loss.item():.4f}")

    average_loss = running_loss / total
    accuracy_percentage = 100 * correct / total
    return average_loss, accuracy_percentage

In [13]:
def validate(model, valid_loader, loss_fn, device):
    """
    Evaluate the model on the validation set.

    Args:
        model: The neural network model to evaluate
        valid_loader: DataLoader for validation data
        loss_fn: Loss function
        device: Device to run evaluation on (cuda/cpu)

    Returns:
        tuple: (average_loss, accuracy_percentage) - average loss and accuracy percentage
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    num_batches = len(valid_loader)

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(valid_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if batch_idx == 0 or (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == num_batches:
                print(f"Validation Batch [{batch_idx + 1}/{num_batches}]")

    average_loss = running_loss / total
    accuracy_percentage = 100 * correct / total
    return average_loss, accuracy_percentage

In [18]:
def run_training_epochs(model, train_loader, valid_loader, loss_fn, optimizer, device, num_epochs=25):
    """
    Train the model for multiple epochs with validation.

    Args:
        model: The neural network model to train
        train_loader: DataLoader for training data
        valid_loader: DataLoader for validation data
        loss_fn: Loss function
        optimizer: Optimization algorithm
        device: Device to run training on (cuda/cpu)
        num_epochs: Number of training epochs (default: 25)

    Returns:
        tuple: (model, history) - best model state and training history dict
               with keys 'train_loss', 'train_acc', 'valid_loss', 'valid_acc'
    """

    print("\n" + "="*50)
    print("Training started...")
    print("="*50)

    best_valid_acc = 0.0
    best_model_state = None
    history = {'train_loss': [], 'train_acc': [], 'valid_loss': [], 'valid_acc': []}

    start_time = time.time()

    for epoch in range(num_epochs):
        epoch_start = time.time()

        train_loss, train_acc = train(model, train_loader, loss_fn, optimizer, device)

        valid_loss, valid_acc = validate(model, valid_loader, loss_fn, device)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['valid_loss'].append(valid_loss)
        history['valid_acc'].append(valid_acc)

        if valid_acc > best_valid_acc:
            best_valid_acc = valid_acc
            best_model_state = copy.deepcopy(model.state_dict())

        epoch_time = time.time() - epoch_start

        print(f"\nEpoch [{epoch+1}/{num_epochs}] | {epoch_time:.2f}s")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Validation Loss:   {valid_loss:.4f} | Validation Accuracy:   {valid_acc:.2f}%")
        print(f"  Best Validation Accuracy: {best_valid_acc:.2f}%")

    total_time = time.time() - start_time
    print(f"\n{'='*50}")
    print(f"Training Complete!")
    print(f"Total time: {total_time/60:.2f} minutes")
    print(f"Best validation accuracy: {best_valid_acc:.2f}%")
    print(f"{'='*50}")

    model.load_state_dict(best_model_state)

    return model, history

In [19]:
num_epochs = 25
best_model_state, train_history = run_training_epochs(model, train_dataloader, valid_dataloader, loss_fn, optimizer, device, num_epochs)


Training started...
Train Batch [1/205] | Loss: 4.6227
Train Batch [10/205] | Loss: 4.4200
Train Batch [20/205] | Loss: 4.3577
Train Batch [30/205] | Loss: 4.4392
Train Batch [40/205] | Loss: 4.4560
Train Batch [50/205] | Loss: 4.0577
Train Batch [60/205] | Loss: 4.2162
Train Batch [70/205] | Loss: 3.8763
Train Batch [80/205] | Loss: 4.0245
Train Batch [90/205] | Loss: 3.2552
Train Batch [100/205] | Loss: 3.5894
Train Batch [110/205] | Loss: 3.4817
Train Batch [120/205] | Loss: 3.1158
Train Batch [130/205] | Loss: 2.4124
Train Batch [140/205] | Loss: 2.4172
Train Batch [150/205] | Loss: 2.3646
Train Batch [160/205] | Loss: 2.8170
Train Batch [170/205] | Loss: 2.6921
Train Batch [180/205] | Loss: 2.2100
Train Batch [190/205] | Loss: 2.5824
Train Batch [200/205] | Loss: 1.9612
Train Batch [205/205] | Loss: 1.7192
Validation Batch [1/26]
Validation Batch [10/26]
Validation Batch [20/26]
Validation Batch [26/26]

Epoch [1/25] | 42.33s
  Train Loss: 3.3396 | Train Acc: 24.73%
  Validation 

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [21]:
# TODO: Do validation on the test set

def test_model(model, test_loader, loss_fn, device):
    """
    Evaluate the model on the test set.

    Args:
        model: The neural network model to test
        test_loader: DataLoader for test data
        loss_fn: Loss function
        device: Device to run test on (cuda/cpu)

    Returns:
        tuple: (average_loss, accuracy_percentage) - average loss and accuracy percentage
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    num_batches = len(test_loader)

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if batch_idx == 0 or (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == num_batches:
                print(f"Test Batch [{batch_idx + 1}/{num_batches}]")

    average_loss = running_loss / total
    accuracy_percentage = 100 * correct / total
    return average_loss, accuracy_percentage


In [23]:
test_loss, test_acc = test_model(model, test_dataloader, loss_fn, device)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.2f}%")

Test Batch [1/26]
Test Batch [10/26]
Test Batch [20/26]
Test Batch [26/26]
Test Loss: 0.1912
Test Accuracy: 94.99%


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [27]:
# TODO: Save the checkpoint
checkpoint = {
    "arch": "resnet50",
    "framework": "pytorch",
    "torchvision_version": torchvision.__version__,
    "in_features": num_features,
    "num_classes": num_classes,
    "epochs_trained": num_epochs,
    "model_state_dict": best_model_state.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "class_to_idx": train_dataset.class_to_idx,
    "class_names": list(train_dataset.class_to_idx.keys()),
}

torch.save(checkpoint, f"flower_classifier.pth")


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [29]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def rebuild_model(path: str, device: str = "cpu"):
    """
    Load the trained flower classification model from a PyTorch checkpoint.

    This function:
    - Reconstructs the ResNet-50 base architecture
    - Rebuilds the custom fully-connected classifier head
    - Loads trained model weights
    - Returns the model in eval mode along with class metadata

    Parameters
    ----------
    path : str
        Path to the saved checkpoint file (e.g. "flower_classifier.pth").

    device : str, optional
        Device to load the model onto ("cpu" or "cuda").
        Default is "cpu".

    Returns
    -------
    model : torch.nn.Module
        The reconstructed model with trained weights loaded,
        set to evaluation mode.

    class_to_idx : dict
        Mapping from class labels (folder names) to integer indices,
        as used during training.

    checkpoint : dict
        The full checkpoint dictionary, containing additional metadata
        such as epochs trained, architecture info, etc.

    Notes
    -----
    - Initial pretrained weights do NOT matter because the trained
      `state_dict` fully overwrites model parameters.
    - The caller is responsible for applying the SAME validation/test
      transforms used during training.
    """
    checkpoint = torch.load(path, map_location=device)

    model = resnet50(weights=None)

    model.fc = nn.Sequential(
        nn.Linear(checkpoint["in_features"], 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, checkpoint["num_classes"]),
    )

    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)
    model.eval()

    return model, checkpoint["class_to_idx"], checkpoint

loaded_model, class_to_idx, model_checkpoint = rebuild_model(path="flower_classifier.pth", device=device)

loaded_model_test_loss, loaded_modeltest_acc = test_model(loaded_model, test_dataloader, loss_fn, device)
print(f"Test Loss: {loaded_model_test_loss:.4f}")
print(f"Test Accuracy: {loaded_modeltest_acc:.2f}%")


Test Batch [1/26]
Test Batch [10/26]
Test Batch [20/26]
Test Batch [26/26]
Test Loss: 0.1912
Test Accuracy: 94.99%


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network.

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training.

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation.

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''

    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()

    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))

    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)

    ax.imshow(image)

    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''

    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='https://github.com/luisjfeliu/python-flower-species-image-classifier/blob/main/assets/inference_example.png?raw=1' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes

## Reminder for Workspace users
If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again.
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.

In [ ]:
# TODO remove .pth files or move it to a temporary `~/opt` directory in this Workspace
!rm checkpoint.pth